<a href="https://colab.research.google.com/github/MONOUE/Project/blob/main/utils_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tslearn

In [ ]:
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans
from yellowbrick.cluster import KElbowVisualizer
from tables.file import defaultdict
import torch
import torch.nn as nn

In [ ]:
def load_stock_data(path_data,path_compagnie):
  compagnie=pd.read_csv(path_compagnie)
  stocks = pd.read_csv(path_data)
  stocks['Date']=pd.to_datetime(stocks['Date'], yearfirst = True)
  names=set(stocks['Symbol'].tolist())
  names=dict.fromkeys(names, 0)
  dataset_global= defaultdict(lambda:0)
  for stock in sorted(names):
    dataset_global[stock]=stocks[stocks['Symbol']==stock].loc[:,["Date","Adj Close","Close","High","Low","Open"]]

  final_names=defaultdict(lambda:0)
  for stock in sorted(names):
    nbr=dataset_global[stock]['Adj Close'].isnull().sum()
    if nbr==0:
      final_names[stock]=0

  market={"NMS":[],"NYQ":[],'NGM':[],'BTS':[]}

  # delete stock that I can't identify exchange market
  delt=0
  for stock in sorted(final_names):
    if compagnie[compagnie.Symbol==stock]['Exchange'].empty:
      final_names.pop(stock)
      delt+=1
    else:
      market[(compagnie[compagnie.Symbol==stock]['Exchange'].values)[0]].append(stock)

  dataset= defaultdict(lambda:0)

  for stock in sorted(final_names):
    dataset[stock]=pd.concat([dataset_global[stock].loc[:,['Date',"Adj Close","Close","High","Low","Open"]],((pd.DataFrame(dataset_global[stock].loc[:,"Adj Close"]).pct_change(periods=1)+1)*100).rename(columns={'Adj Close':'Adj_pct'})],axis=1, join='inner')

  return (dataset,market,final_names)

In [ ]:
def build_feature(data,final_names):
  mov_avg_close_5=defaultdict(lambda:0)
  mov_avg_close_12=defaultdict(lambda:0)
  mov_avg_close_26=defaultdict(lambda:0)
  mov_avg_close_30=defaultdict(lambda:0)
  mov_avg_close_50=defaultdict(lambda:0)

  for stock in sorted(final_names):
    mov_avg_close_5[stock]=pd.concat([data[stock]['Date'],data[stock].loc[:,'Adj Close':'Open'].rolling(5).mean().rename(columns={'Adj Close':'Adj_mvg5','Close':'Close_mvg5','High':'High_mvg5','Low':'Low_mvg5','Open':'Open_mvg5'})],axis=1, join='inner')
    mov_avg_close_12[stock]=pd.concat([data[stock]['Date'],data[stock].loc[:,'Adj Close':'Open'].rolling(12).mean().rename(columns={'Adj Close':'Adj_mvg12','Close':'Close_mvg12','High':'High_mvg12','Low':'Low_mvg12','Open':'Open_mvg12'})],axis=1, join='inner')
    mov_avg_close_26[stock]=pd.concat([data[stock]['Date'],data[stock].loc[:,'Adj Close':'Open'].rolling(26).mean().rename(columns={'Adj Close':'Adj_mvg26','Close':'Close_mvg26','High':'High_mvg26','Low':'Low_mvg26','Open':'Open_mvg26'})],axis=1, join='inner')
    mov_avg_close_30[stock]=pd.concat([data[stock]['Date'],data[stock].loc[:,'Adj Close':'Open'].rolling(30).mean().rename(columns={'Adj Close':'Adj_mvg30','Close':'Close_mvg30','High':'High_mvg30','Low':'Low_mvg30','Open':'Open_mvg30'})],axis=1, join='inner')
    mov_avg_close_50[stock]=pd.concat([data[stock]['Date'],data[stock].loc[:,'Adj Close':'Open'].rolling(50).mean().rename(columns={'Adj Close':'Adj_mvg50','Close':'Close_mvg50','High':'High_mvg50','Low':'Low_mvg50','Open':'Open_mvg50'})],axis=1, join='inner')

  for stock in sorted(final_names):
    data[stock]=data[stock].merge(mov_avg_close_5[stock],how='inner', on='Date')
    data[stock]=data[stock].merge(mov_avg_close_12[stock],how='inner', on='Date')
    data[stock]=data[stock].merge(mov_avg_close_26[stock],how='inner', on='Date')
    data[stock]=data[stock].merge(mov_avg_close_30[stock],how='inner', on='Date')
    data[stock]=data[stock].merge(mov_avg_close_50[stock],how='inner', on='Date')

  return data

In [ ]:
def stock_rank(data,market_list,market_name):
  stock_return=data[sorted(market_list[market_name])[0]].loc[:,['Date','Adj_pct']].rename(columns={'Adj_pct':sorted(market_list[market_name])[0]+"_rt"})
  data[sorted(market_list[market_name])[0]].drop(columns=['Adj_pct'],inplace=True)
  for stock in sorted(market_list[market_name])[1:]:
    stock_return=stock_return.merge(data[stock].loc[:,['Date','Adj_pct']].rename(columns={'Adj_pct':stock+"_rt"}),how='inner',on='Date')
    data[stock].drop(columns=['Adj_pct'],inplace=True)

  rank_stock=pd.concat([stock_return["Date"].iloc[1:],stock_return.iloc[1:,:].rank(axis=1,method="min")],axis=1,join="inner")
  return (rank_stock,stock_return)


In [ ]:
def final_data(data,market_list,market_name,rank_stock):
  torch_data=torch.from_numpy(data[sorted(market_list[market_name])[0]].iloc[1:,:].merge(rank_stock[0].loc[:,['Date',sorted(market_list[market_name])[0]+'_rt']],how='inner', on='Date').iloc[:,1:].values.T).unsqueeze(0)
  for stock in sorted(market_list[market_name][1:]):
    torch_data= torch.cat([torch_data,torch.from_numpy(data[stock].iloc[1:,:].merge(rank_stock[0].loc[:,['Date',stock+'_rt']],how='inner', on='Date').iloc[:,1:].values.T).unsqueeze(0)],0)
  stock_rt=(rank_stock[1].iloc[1:,1:].values).T
  return (torch_data, stock_rt)

In [ ]:
def split_datasets_for_training(data,r_data, window_size,scale=True):
  _l = data.shape[-1]
  Xs = []
  Ys = []
  Rs=[]
  for i in range(48, (_l - window_size-1)):
    Xs.append(data[:,:-1,i:i+window_size])
    # Ys.append(data[:,-1,i+window_size+1])
    Ys.append(r_data[:,i+window_size+1])
    x=r_data[:,i:i+window_size]
    # y=r_data[:,i+window_size+1]
    Rs.append(x)

  # tr_x, ts_x, tr_y, ts_y,tr_r,ts_r,tr_s,ts_s = train_test_split(Xs, Ys,Rs,y,shuffle=False)
  tr_x, ts_x, tr_y, ts_y,tr_r,ts_r = train_test_split(Xs,Ys,Rs,shuffle=False)

  # return 1
  return  (tr_x, ts_x, tr_y, ts_y,tr_r,ts_r)

In [ ]:
class build_cluster():
  '''
  'silhouette','calinski_harabasz',distortion

  '''
  def __init__(self, metric: str,KElbowmetric:str, iteration: int, num_clus:int=0) :
    super(build_cluster, self).__init__()
    self.Kmeans_metric=metric
    self.metric=KElbowmetric
    self.num_cluster=num_clus
    self.iter=iteration
    self.model= TimeSeriesKMeans(metric= self.Kmeans_metric, max_iter=self.iter)

  def predict(self,data,k=(2,10),best=True):
    # visualizer = KElbowVisualizer(self.model, k=k,metric=self.metric, timings= True)
    # visualizer.fit(np.array(data))
    # if best and visualizer.elbow_value_ !=None:
    #   self.num_cluster= visualizer.elbow_value_
    self.num_cluster=8
    model=TimeSeriesKMeans(n_clusters=self.num_cluster,metric= self.Kmeans_metric, max_iter=self.iter).fit(np.array(data))
    label=model.predict(np.array(data))
    return label

In [ ]:
class Adjacency_Matrix:
    def __init__(self, market_name):
        # self.data_path = data_path
        # self.date_format = '%Y-%m-%d'
        self.market_name = market_name

    def generate_stock_relation(self, market_dict,
                                 pred_clusters,lap=False):

        selected_stock=sorted(market_dict[self.market_name])
        stock_index = {}

        for index, ticker in enumerate(selected_stock):
            stock_index[ticker] = index

        valid_label_count = 0
        valid_label_index = {}

        for label in np.unique(pred_clusters):


          if np.where(pred_clusters==label)[0].shape[0]>1:
              valid_label_index[str(label)] = valid_label_count
              valid_label_count += 1

        one_hot_label_embedding = np.identity(valid_label_count,
                                                 dtype=float)

        stock_relation_embedding = np.zeros(
            [len(selected_stock), len(selected_stock),
             valid_label_count], dtype=float)



        for label in valid_label_index.keys():
            cur_ind_stock = np.where(pred_clusters==int(label))
            if cur_ind_stock[0].shape[0] <= 1:
                print('shit label:', label)
                continue
            ind_ind = valid_label_index[label]


            for i in range(cur_ind_stock[0].shape[0]):

              left_tic_ind = cur_ind_stock[0][i]

              stock_relation_embedding[left_tic_ind][left_tic_ind] = copy.copy(one_hot_label_embedding[ind_ind])



              for j in range(i + 1, cur_ind_stock[0].shape[0]):

                right_tic_ind = cur_ind_stock[0][j]
                stock_relation_embedding[left_tic_ind][right_tic_ind] = copy.copy(one_hot_label_embedding[ind_ind])
                stock_relation_embedding[right_tic_ind][left_tic_ind] = copy.copy(one_hot_label_embedding[ind_ind])


        adj=torch.from_numpy(np.sum(stock_relation_embedding,axis=2)).to_sparse()

        # compute the weight of edges


        return adj


In [ ]:
def get_edges(adj):
  ''' This function is used to get edges of the adjacency matrix
  '''
  return adj.indices()

  # adj_edges=[adj.indices()[:,i].tolist() for i in range(adj.indices().shape[-1])]
  # return torch.tensor(adj_edges).t().contiguous()

In [ ]:
def batch_data(x,y,predict_cluster,market_dict,market_name,batch_size=32,seq_len=60):
    # Define the batch size and the sequence length
    batch_adj=[]
    batch_x=[]
    batch_y=[]
    Adj=Adjacency_Matrix(market_name)

    # Define the number of nodes and the number of edges
    num_nodes = len(market_dict[market_name])
    # num_edges = 2

    num_batch=(len(x)//batch_size)
    for k in range(num_batch):

      batch_x.append(torch.cat([x[ind].unsqueeze(0).float()  for ind in range(k*batch_size,(k+1)*batch_size)],axis=0))
      # batch_y.append(torch.cat([y[ind].unsqueeze(0).float()  for ind in range(k*batch_size,(k+1)*batch_size)],axis=0))
      batch_y.append(torch.cat([torch.from_numpy(y[ind]).unsqueeze(0).float()  for ind in range(k*batch_size,(k+1)*batch_size)],axis=0))



      cluster=predict_cluster[k*batch_size:(k+1)*batch_size]

      # Initialize the empty adjacency matrix
      # adj = torch.zeros(batch_size, seq_len, num_nodes,num_nodes)
      adj = torch.zeros(batch_size, num_nodes,num_nodes)


      # Fill in the adjacency matrix for each time step in each batch
      for i in range(batch_size):
          # Define the adjacency matrix for a single time step
          adj_t = Adj.generate_stock_relation(market_dict,cluster[i]).to_dense()
          A_hat= adj_t+torch.eye(adj_t.shape[0])
          D=torch.linalg.inv(torch.diag(torch.sum(A_hat,axis=1)))**(0.5)
          adj_norm= torch.mm(torch.mm(D,A_hat),D)
          adj[i,:, :] = copy.copy(adj_norm)
          # for j in range(seq_len):

              # adj_t = adj_t   # Add a new dimension for the
              # adj_t = adj_t.unsqueeze(0).unsqueeze(0).repeat(1, 1, 1, 1, num_edges)   # Add a new dimension for the edges
              # adj[i, j, :, :] = copy.copy(adj_t)
      batch_adj.append(adj)
    return batch_x,batch_y,batch_adj



In [ ]:
def ndcg_score(targets, scores, k=127):

    scores_predict=torch.argsort(torch.argsort(scores,descending=True, dim=1),dim=1)
    score_real=torch.argsort(torch.argsort(targets,descending=True, dim=1),dim=1)

    relevance=torch.where(scores_predict<50,1,0) # predicted relevance
    relevance_real=torch.where(score_real<50,1,0) #real relevance

    _, indices = torch.topk(targets, k, dim=1, largest=True, sorted=True)# find the index of the top k relevant stock

    sorted_targets = torch.gather(relevance, 1, indices)# use that index to get their predicted relevance score
    discounts = torch.log2(torch.arange(2, k + 2).float())# compute de discount
    gains = (2 ** sorted_targets - 1) / discounts

    ideal_sorted_targets = torch.gather(relevance_real, 1, indices)# use the index of top k relevant to get their true relevance score
    ideal_gains = (2 ** ideal_sorted_targets - 1) / discounts

    dcg = torch.sum(gains, dim=1)
    ideal_dcg = torch.sum(ideal_gains, dim=1)

    ndcg = torch.mean(dcg/ideal_dcg)
    return ndcg